In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import doctest
import implicit
import sklearn
from sklearn.model_selection import train_test_split
import scipy
from scipy.sparse import csr_matrix
import pandas as pd
from gensim import models
from gensim import similarities
from gensim.models import Word2Vec

In [2]:
MOCK = True

In [3]:
if MOCK:
    pd_data = pd.read_csv("./data/ratings_mock.csv")
else:
    pd_data = pd.read_csv("./data/ratings.csv")

In [4]:
data = pd_data

# Data

matrix.data < 4 используются для того, чтобы превратить данные из явных в неявные.

In [5]:
def to_sparce_matrix(pd_data):
    '''
    >>> to_sparce_matrix(pd.DataFrame({'rating': [3,5], 'userId' : [1, 2], 'movieId': [1, 2]})).toarray()
    array([[0., 0., 0.],
           [0., 0., 0.],
           [0., 0., 1.]])
    '''
    matrix = csr_matrix((pd_data['rating'].values, (pd_data['userId'].values, pd_data['movieId'].values)))
    matrix.data[matrix.data < 4] = 0
    matrix.eliminate_zeros()
    matrix.data = np.ones(len(matrix.data))
    return matrix

# Init

In [6]:
k = 2
LAMBDA = 0.001
ALPHA = 0.001
COEF_FOR_OPTIMIZATION = 0.1
NUM_OF_NEAREST = 5

In [7]:
p = None
q = None
b_u = None
b_i = None
r = None

In [8]:
def init():
    global p, q, b_u, b_i, r, U, I
    r = to_sparce_matrix(data)
    U, I = r.shape
    p = [np.random.rand(k) for u in range(U)]
    q = [np.random.rand(k) for i in range(I)]
    b_u = np.random.rand(U)
    b_i = np.random.rand(I)

In [9]:
init()

# Similarity

## Global model

In [10]:
def get_corpus():
    # MOCK
    # ['1', '2']
    # ['2', '3']
    corpus = [[str(i)] for i in range(I)]
    for u in range(U):
        frist, second = r[u].nonzero()
        corpus.append(list(map(str, second)))
    return corpus

In [11]:
corpus = get_corpus()

In [12]:
model_word2vec = Word2Vec(sentences=corpus, size=100, window=5, min_count=1, workers=4)
word_vec = model_word2vec.wv

In [13]:
def s(i, j):
    '''
        # MOCK return 0
    '''
    return word_vec.similarity(str(i), str(j))

In [14]:
def get_k_nearest(index):
    '''
    '''
    #   MOCK return [index] * NUM_OF_NEAREST
    result = list(map(lambda x: int(x[0]), word_vec.most_similar(str(index))[:NUM_OF_NEAREST]))
    return result

# Algo

In [15]:
def dot_product(x):
    return np.dot(x, x.T)

In [16]:
def all_dot_product(x):
    return sum(dot_product(e) for e in x)

In [17]:
def r_(u, i):
    return np.dot(p[u], q[i].T)

In [18]:
def first_sum():
    value_from_r = 0
    for u in range(U):
        for i in range(I):
            value_from_r += (r[u,i] - r_(u, i)) ** 2

    return value_from_r

In [19]:
def second_sum(): 
    value = 0
    for i in range(I):
        for j in get_k_nearest(i):
            value += (s(i, j) - np.dot(q[i], q[j].T)) ** 2
    return value

In [20]:
def omega():
    return all_dot_product(p) + all_dot_product(q) + \
           dot_product(b_u) + dot_product(b_i)

In [21]:
def function():
    return 0.5 * first_sum() + \
           ALPHA * 0.5 * second_sum() + \
           LAMBDA * 0.5 * omega()

In [22]:
def dfdp(): 
    list_dp = list()
    for u in range(U):
        first = sum((r_(u, i) - r[u, i]) * q[i] for i in range(I))
        second = LAMBDA * p[u]
        list_dp.append(first + second)
    return list_dp

In [23]:
def dfdq():
    list_dq = list()
    for i in range(I):
        first = sum((r_(u, i) - r[u, i]) * p[u] for u in range(U))
        second = -ALPHA * \
                 sum((s(i, j) - np.dot(q[i], q[j].T) + LAMBDA) * q[j] \
                     for j in get_k_nearest(i))
            
        list_dq.append(first + second)
    return list_dq

In [ ]:
def dfdb_u():
    delta = 0
    for u in range(U):
        for i in range(I):
            delta += r_(u, i) - r[u, i]
    return delta + LAMBDA * b_u

In [ ]:
def dfdb_i():
    delta = 0
    for u in range(U):
        for i in range(I):
            delta += r_(u, i) - r[u, i]
    return delta + LAMBDA * b_i

In [ ]:
def optimize_u():
    _dfdp = dfdp()
    _dfdb_u = dfdb_u()
    for u in range(U):
        p[u] -= _dfdp[u] * COEF_FOR_OPTIMIZATION
        b_u[u] -= _dfdb_u[u] * COEF_FOR_OPTIMIZATION

In [ ]:
def optimize_i():
    _dfdq = dfdq()
    _dfdb_i = dfdb_i()
    for i in range(I):
        q[i] -= _dfdq[i] * COEF_FOR_OPTIMIZATION
        b_i[i] -= _dfdb_i[i] * COEF_FOR_OPTIMIZATION

In [ ]:
def optimize():  
    optimize_u()
    optimize_i()

# Start

In [ ]:
for step in range(50):
    value = function()
    print("step: {}, value: {}".format(step, value))
    optimize()  

step: 0, value: 50199.93823200982
step: 1, value: 2.581519638777576e+26
